In [5]:
import os 
import pandas as pd 
import numpy as np 
exp_name = "exp_6_2_debising"
reasonings = [
    "fewshot_0_0",
    "fewshot_0_1",
    "fewshot_0_2",
    "fewshot_1_0",
    "fewshot_1_1",
    "fewshot_1_2",
    "fewshot_2_0",
    "fewshot_2_1",
    "fewshot_2_2",
    "logicshot_2_2",
]
new_dfs = []
for reasoning in reasonings:
    dir_path = f"results/{exp_name}/{reasoning}"
    models = [
        'openai--gpt-4o-mini', 
        'llama3_1_instruct--70b', 
        'llama3_1--8b', 
        'gemma2--9b', 
        # 'exaone--8b', 
        'qwen2--7b'
        ]
    result_dfs = []
    for model in models:
        result_df = pd.read_csv(os.path.join(dir_path, f"{model}.csv"))
        result_df['model'] = model
        result_dfs.append(result_df)

    def mv_func(x):
        values = ['must', 'should', 'ought']
        for v in values:
            if v in x:
                return v 
        if "has to" in x or "have to" in x:
            return "have to"
        return "none"

    def process_model_output(x):
        x = str(x).strip()
        # return either 1 or 0 first appearance
        for i in range(len(x)):
            if x[i] == "1" or x[i] == "0":
                return int(x[i])
        return  np.nan #  -100000 # np.nan
        
        # x = str(x).strip()
        # if x.count("Answer:") > 1:
        #     # drop when the second answer appears
        #     x = x[:x.find("Answer:")+len("Answer:")]
        # elif "Answer:" in x:
        #     x = x.split("Answer:")[1][:3]
        # else:
        #     x = x[:3]
        # x =str(x)
        # if "1" in x and "0" in x:
        #     return np.nan
        # elif "1" in x:
        #     return 1
        # elif "0" in x:
        #     return 0
        # else:
        #     return np.nan 
    result_df = pd.concat(result_dfs)
    result_df['mv'] = result_df['input'].apply(mv_func)
    result_df['model_output'] =  result_df['model_output_raw'].apply(process_model_output)
    result_df.head()

    targets = ['input_type', 'model', 'model_output']
    groupby = targets[:-1]
    result_averaged = result_df[targets].groupby(groupby).mean().reset_index()
    result_averaged
    new_df = pd.DataFrame(columns=['type'] + models)
    for input_type in ['strong',]:
        new_row = [input_type]
        for model in models:
            temp = result_averaged[result_averaged['input_type'] == input_type]
            temp = temp[temp['model'] == model]
            new_row.append(np.round(temp['model_output'].item(), 2))
        new_df.loc[len(new_df)] = new_row
    new_df['reasoning'] = reasoning
    new_dfs.append(new_df)

new_dfs = pd.concat(new_dfs)
new_dfs.to_csv(f'trainging_free_fewshot_{exp_name}.csv', index=False)

In [6]:
new_dfs

,type,openai--gpt-4o-mini,llama3_1_instruct--70b,llama3_1--8b,gemma2--9b,qwen2--7b,reasoning
0,strong,0.83,0.98,0.90,0.98,0.94,fewshot_0_0
0,strong,0.73,0.57,0.71,0.90,0.69,fewshot_0_1
0,strong,0.82,0.55,0.64,0.77,0.70,fewshot_0_2
0,strong,0.60,0.60,0.07,0.64,0.78,fewshot_1_0
0,strong,0.63,0.33,0.37,0.67,0.75,fewshot_1_1
0,strong,0.71,0.37,0.52,0.65,0.72,fewshot_1_2
0,strong,0.53,0.60,0.04,0.71,0.67,fewshot_2_0
0,strong,0.63,0.22,0.17,0.40,0.77,fewshot_2_1
0,strong,0.58,0.33,0.35,0.36,0.78,fewshot_2_2
0,strong,0.48,0.19,0.30,0.12,0.47,logicshot_2_2
